## TLDR

A simple example (sentiment analysis task)

In [1]:
import t2t

In [2]:
trainer_arguments = t2t.TrainerArguments(
    # model
    model_name_or_path="distilgpt2",
    cache_dir="/workspace/cache",
    # data inputs
    train_file="../sample_data/trainlines.txt",
    validation_split=0.1,
    block_size=512,
    # taining outputs
    output_dir="/tmp/saved_model",
    overwrite_output_dir=True,
    # training settings
    num_train_epochs=3,
    per_device_train_batch_size=24,
    gradient_accumulation_steps=1,
    learning_rate=1e-5,
    gradient_checkpointing=False,
    adafactor=False,
    fp16=True,
    # validation settings
    per_device_eval_batch_size=24,
    evaluation_strategy="epoch",
)
trainer = t2t.Trainer(arguments=trainer_arguments)
#trainer.freeze(embeddings=True)

Training causal Language Model
Loading distilgpt2 (for large models, this might take a while)
Files will be cached at: /workspace/cache
Ensure this directory is persistent if you do not want to download model files again!
Loading text dataset
keep_linebreaks True


Using custom data configuration default-0bcde2b719600dec
Reusing dataset text (/workspace/cache/text/default-0bcde2b719600dec/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5)


  0%|          | 0/1 [00:00<?, ?it/s]

Auto-split validation set from training set: 10.0 %


Using custom data configuration default-57e036a714c1aab0
Reusing dataset text (/workspace/cache/text/default-57e036a714c1aab0/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5)
Loading cached split indices for dataset at /workspace/cache/text/default-57e036a714c1aab0/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5/cache-9f15afa515bf754a.arrow and /workspace/cache/text/default-57e036a714c1aab0/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5/cache-8dc6ba7fa3584620.arrow


Distributed training: False


In [3]:
trainer.model_summary()

Summary
- model name: distilgpt2
- model params:
  - train: 81.9 M
  - total: 81.9 M
  - vocab: 50257
- prompt tuning only: False


### Train Model

In [4]:
trainer.train(valid=True)

Running tokenizer on dataset:   0%|          | 0/8 [00:00<?, ?ba/s]

Running tokenizer on dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

Grouping texts in chunks of 512:   0%|          | 0/8 [00:00<?, ?ba/s]

Grouping texts in chunks of 512:   0%|          | 0/1 [00:00<?, ?ba/s]

Using amp fp16 backend
***** Running training *****
  Num examples = 3279
  Num Epochs = 3
  Instantaneous batch size per device = 24
  Total train batch size (w. parallel, distributed & accumulation) = 24
  Gradient Accumulation steps = 1
  Total optimization steps = 411


Epoch,Training Loss,Validation Loss
1,3.959400,3.808030
2,3.873800,3.790116
3,3.857100,3.785381


***** Running Evaluation *****
  Num examples = 367
  Batch size = 24
Saving model checkpoint to /tmp/saved_model/checkpoint-137
Configuration saved in /tmp/saved_model/checkpoint-137/config.json
Model weights saved in /tmp/saved_model/checkpoint-137/pytorch_model.bin
tokenizer config file saved in /tmp/saved_model/checkpoint-137/tokenizer_config.json
Special tokens file saved in /tmp/saved_model/checkpoint-137/special_tokens_map.json
Deleting older checkpoint [/tmp/saved_model/checkpoint-26] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 367
  Batch size = 24
Saving model checkpoint to /tmp/saved_model/checkpoint-274
Configuration saved in /tmp/saved_model/checkpoint-274/config.json
Model weights saved in /tmp/saved_model/checkpoint-274/pytorch_model.bin
tokenizer config file saved in /tmp/saved_model/checkpoint-274/tokenizer_config.json
Special tokens file saved in /tmp/saved_model/checkpoint-274/special_tokens_map.json
Deleting older checkpoint [/tmp/sa

***** train metrics *****
  epoch                    =        3.0
  total_flos               =  1196924GF
  train_loss               =     3.8968
  train_runtime            = 0:02:23.96
  train_samples            =       3279
  train_samples_per_second =      68.33
  train_steps_per_second   =      2.855


### Test Model

In [5]:
input_text = "<start> This is the worst movie I have ever seen! <end> SENTIMENT: "
trainer.generate_single(input_text, max_length=128).split(" <end><start>")[0]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'<start> This is the worst movie I have ever seen! <end> SENTIMENT: ***********'

In [6]:
input_text = "<start> This is the best movie I have ever seen! <end> SENTIMENT: "
trainer.generate_single(input_text, max_length=128).split(" <end><start>")[0]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'<start> This is the best movie I have ever seen! <end> SENTIMENT: ***********'

## 